In [ ]:
!pip install pyspellchecker
!pip install sentence-transformers
!pip install pprint
!pip install scipy
!pip install afinn

     |████████████████████████████████| 1.9MB 8.1MB/s 
     |████████████████████████████████| 71kB 4.0MB/s 
     |████████████████████████████████| 778kB 12.6MB/s 
     |████████████████████████████████| 890kB 27.8MB/s 
     |████████████████████████████████| 3.0MB 22.1MB/s 
     |████████████████████████████████| 1.1MB 52.8MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.2-cp36-none-any.whl size=93964 sha256=3cabd4b1d2f3c2666db2d33a6405ddf189f3df030257e1a8bd1d037ef5ffae98
  Stored in directory: /root/.cache/pip/wheels/f7/06/a0/567f3651876165429f6510d3197b011652a25e547552816824
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=b9e961857afcde07c3bf35f8d3ba41eff112f7ecc0acd270d4c040a5c98ba458
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses
  Created wheel for pprint: filename=pprint-0.1-cp36-none-a

In [ ]:
import os
import glob
import re
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import string
from spellchecker import SpellChecker
import nltk
nltk.download('stopwords')#Error loading
nltk.download('punkt')#Error loading
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer

from nltk.tokenize import word_tokenize 
from nltk import word_tokenize, pos_tag
from nltk.stem import SnowballStemmer
from sklearn.decomposition import TruncatedSVD, LatentDirichletAllocation
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

import pickle
from sklearn.cluster import KMeans

import gensim
from gensim import corpora
import scipy as scipy

import spacy
from spacy import displacy
import en_core_web_sm

from wordcloud import WordCloud, STOPWORDS 
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px

import warnings
warnings.filterwarnings('ignore')
from sentence_transformers import SentenceTransformer
import torch
print(torch.cuda.is_available())

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


False


In [ ]:
path = '/content/drive/My Drive/Hackathon_2020/data'

In [ ]:
df_geotagged = pd.read_csv(os.path.join(path, 'all_tweets_usa.csv'),
                           header = 0, sep = ',', error_bad_lines=False, encoding = "ISO-8859-1", parse_dates = True)
df_geotagged = df_geotagged.drop(['Unnamed: 0'], axis = 1)
df_geotagged.drop_duplicates(inplace=True)
df_geotagged = df_geotagged[df_geotagged['text'].notna()]
df_geotagged['is_retweet'] = df_geotagged['text'].apply(lambda x: x[:2]=='RT')
print("Total number of Retweets: {}".format(df_geotagged['is_retweet'].sum()))
df_geotagged

FileNotFoundError: ignored

In [ ]:
df_geotagged.columns
df_geotagged['user_screen_name'].value_counts()

VirallyAds         2170
pairsonnalitesN    1764
stigmabase          676
Weeklyvoice         415
johnnybebad666_     388
                   ... 
dodger1               1
almostanythingi       1
melissafreyauth       1
joejoetill            1
cupcakesally          1
Name: user_screen_name, Length: 26396, dtype: int64

In [ ]:
df_news = pd.read_csv(os.path.join(path, 'news_org_summary_2018.csv'), header = 0, sep = ',')                          
#print(df_news['country.query'].value_counts())
df_news = df_news[df_news['country.query'] == 'USA_en']
newscreen_names = list(df_news['user_screen_name'].values)
newscreen_names

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
df_geotagged.sort_values(['user_followers_count'], ascending = False)['user_screen_name'].value_counts().to_csv(os.path.join(path, 'ieee_geotagged_user_screen_names.csv'))

In [ ]:
df_geotagged[df_geotagged['user_screen_name.1'].isin(newscreen_names)]

,coordinates,created_at,hashtags,media,urls,favorite_count,id,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_user_id,lang,place,possibly_sensitive,retweet_count,reweet_id,retweet_screen_name,source,text,tweet_url,user_created_at,user_screen_name,user_default_profile_image,user_description,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_screen_name.1,user_statuses_count,user_time_zone,user_urls,user_verified,is_retweet


In [ ]:
df_geotagged['user_screen_name.1'].value_counts()

VirallyAds         2170
pairsonnalitesN    1764
stigmabase          676
Weeklyvoice         415
johnnybebad666_     388
                   ... 
MarkyNDenverxxx       1
DrJoyArciaga          1
nadiarosemusic        1
LiveGreenTO           1
capati_patrick        1
Name: user_screen_name.1, Length: 26392, dtype: int64

In [ ]:
!pip install newsapi-python

In [ ]:
from newsapi import NewsApiClient

# Init
newsapi = NewsApiClient(api_key="key")

# /v2/top-headlines
# top_headlines = newsapi.get_top_headlines(q='bitcoin',
#                                           sources='bbc-news,the-verge',
#                                           category='business',
#                                           language='en',
#                                           country='us')
# print(top_headlines)


In [ ]:
# /v2/sources
sources = newsapi.get_sources(country= 'us')
print(len(sources))
df_news_api = pd.DataFrame(sources['sources'])
df_news_api.to_csv(os.path.join(path, 'news_api_names.csv'))

NewsAPIException: ignored

In [ ]:
set(df_news_api['name'].values)

NameError: ignored

In [ ]:
# /v2/everything
all_articles = newsapi.get_everything(q='covid',
                                      
                                      from_param='2020-06-24',
                                      to='2020-07-10',
                                      language='en',
                                      sort_by='relevancy',
                                      page=2)
pd.DataFrame(all_articles['articles'])

NameError: ignored

In [ ]:
df_all_ids = pd.read_csv(os.path.join(path, 'all_no_geotagged_tweet_ids.csv'), header = None)
df_all_ids.columns = ['id', 'some_measure']
df_geotagged.drop_duplicates(inplace=True)
df_all_ids = df_all_ids[['id']]
df_all_ids.head(5)

,id
0,1279619193709658112
1,1279619193839607811
2,1279619193915113476
3,1279619194183733251
4,1279619194573606915


In [ ]:
df_all_ids.to_csv(os.path.join(path, 'all_tweets_ieee.csv'), header= None, index = False)

In [ ]:
i =0
with open(os.path.join(path, 'all_tweets_ieee.csv')) as f:
  for line in f.readlines():
    if i <5:
      print(line)
      i +=1 
    else:
      break

1279619193709658112

1279619193839607811

1279619193915113476

1279619194183733251

1279619194573606915



In [ ]:
len(df_all_ids)

51288288

### Data Cleaning

In [ ]:
#Utility function for data preprocessing (METHOD 1)
stop_words = set(stopwords.words('english'))
stop_words.update(['zero','one','two','three','four','five','six','seven','eight','nine','ten','may','also','across','among','beside','however','yet','within'])
re_stop_words = re.compile(r"\b(" + "|".join(stop_words) + ")\\W", re.I)
stemmer = SnowballStemmer("english")

def cleanHtml(sentence):
    cleanr = re.compile(r"http\S+")
    cleantext = re.sub(cleanr, ' ', str(sentence))
    return cleantext
def cleanPunc(sentence): #function to clean the word of any punctuation or special characters
    cleaned = re.sub(r'[?|!|\'|"|#]',r'',sentence)
    cleaned = re.sub(r'[.|,|)|(|\|/]',r' ',cleaned)
    cleaned = cleaned.strip()
    cleaned = cleaned.replace("\n"," ")
    return cleaned
def keepAlpha(sentence):
    alpha_sent = ""
    for word in sentence.split():
        alpha_word = re.sub('[^a-z A-Z]+', ' ', word)
        alpha_sent += alpha_word
        alpha_sent += " "
    alpha_sent = alpha_sent.strip()
    return alpha_sent
def removeStopWords(sentence):
    global re_stop_words
    return re_stop_words.sub(" ", sentence)

def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stem = stemmer.stem(word)
        stemSentence += stem
        stemSentence += " "
    stemSentence = stemSentence.strip()
    return stemSentence
def replace_mentioned(tweet):
        '''This function will extract the twitter handles of people mentioned in the tweet'''
        return re.sub('(?<!RT\s)(@[A-Za-z]+[A-Za-z0-9-_]+)',"", tweet)
    
def find_hashtags(sentence):
        '''This function will extract hashtags'''
        return re.sub('(#[A-Za-z]+[A-Za-z0-9-_]+)', "", sentence)
def fix_abbreviation(text):
    text = re.sub(r'\b(?:vs|VS|Vs)\b\.*', 'versus', text)
    text = re.sub(r"'ve\b", ' have', text)
    # text = re.sub(r"\b(btw|BTW)\b", 'by the way', text)
    # text = re.sub(r"\b(asap|ASAP)\b", 'as soon as possible', text)
    # text = re.sub(r"\b(af|AF)\b", 'as fuck', text)
    # text = re.sub(r"\b(AFAIK)\b", 'as far as I know', text)
    text = re.sub(r"\b(DM)\b", 'direct message', text)
    text = re.sub(r"\b(EM)\b", 'email', text)
    text = re.sub(r"\b(FB)\b", 'facebook', text)
    # text = re.sub(r"\b(OMG)\b", 'oh my god', text)
    return text

def remove_extras(text):
    # Remove 'RT'
    text = re.sub(r'RT\b', '', text)
    # Remove user's handle and email addres
    text = re.sub(r'\S*@\S*\s?', '', text)
    # Remove links
    text = re.sub(r'http(s)?\:\/\/t\.co\/\w*', '', text)
    # Remove digits and words that has digit(s) except those network related
    # text = re.sub(r'(?!5g|5G|4g|4G|3g|3G|2g|2G)\w*\d+\w*', '', text)
    text = re.sub(r'\W\b(\d+)\b', '', text)
    # Remove '-'
    text = re.sub(r'-', '', text)
    # Remove distracting double quotes
    text = re.sub(r"\”", '\"', text)
    text = re.sub(r"\“", '\"', text)
    text = re.sub(r"\"", '', text)
    # Remove distracting single quotes
    text = re.sub(r"\’", '\'', text)
    text = re.sub(r"\‘", '\'', text)
    text = re.sub(r"\'s", '', text)
    # text = re.sub(r"\'ve", ' have', text)
    text = re.sub(r"\'", '', text)
    # Remove extra whitespaces
    text = ' '.join([word for word in text.split()])

    return text
def reduce_lengthening(text):
    """
    Fixing Word Lengthening ~ finalllly -> finally
    """
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)
def clean_ascii(text):
    # function to remove non-ASCII chars from data
    return ''.join(i for i in text if ord(i) < 128)

def clean_HT_MT_Html(text):
  text = replace_mentioned(text)
  #text = find_hashtags(text)
  text = cleanHtml(text)
  text = clean_ascii(text)
  # Remove punctuations
  punc_free = ''.join([ch for ch in text if ch not in string.punctuation])

  # Fix misspelled words
  text = ' '.join([spell.correction(word) for word in punc_free.split()])
  text = text.strip()
  return text


In [ ]:
#Utility function for data preprocessing (METHOD 2)

stop_words = stopwords.words('english')
stop_words.extend(['zero','is','lol','yr','fu', 'weeks','im', 'u','one', 'day', 'week', 'two', 'dont', 'it', 'isnt',"isn't", "don't",'not','yes','no','next'])
puncs = string.punctuation
lemma = WordNetLemmatizer()
spell = SpellChecker(distance=1)
# Add unknown words to spell checker
spell.word_frequency.load_words(['Covid', 'covid', 'COVID', 'corona'])

def clean_doc(text):
    # pdb.set_trace()

    # Fix some common abbreviations
    text = fix_abbreviation(text)
    text = cleanHtml(text)
    text = replace_mentioned(text)
    #text = find_hashtags(text)

    # Remove unnecessary tokens like username, email and '-'
    text = remove_extras(text)
    # Remove extra repeated characters

    text = reduce_lengthening(text)
    
    text = keepAlpha(text)
    # Remove punctuations
    punc_free = ''.join([ch for ch in text if ch not in puncs])

    # Fix misspelled words
    spell_corrected = ' '.join([spell.correction(word) for word in punc_free.split()])

    # Lemmatize word using WordNetLemmatizer
    # Define word's role (POS) for better lemmatization result
    lemmaztized_words = []
    for word, pos in nltk.pos_tag(spell_corrected.split()):
        if pos[:2] == 'VB':     # if verb
            tag = 'v'
        elif pos[:2] == 'JJ':   # if adjective
            tag = 'a'
        else:
            tag = 'n'
        lemmaztized_words.append(lemma.lemmatize(word, pos=tag))
    normalized = ' '.join([word for word in lemmaztized_words])

    # Remove stop words
    stop_free = ' '.join([word for word in normalized.lower().split() if word not in stop_words])

    return stop_free

In [ ]:
df = df_geotagged


#### Cleaning Method 1

In [ ]:
df['text_clean'] = df.text.apply(lambda x: clean_doc(x))

#### Cleaning Method 2

In [ ]:
df['text_noHT_MT'] = df.text.apply(lambda x: clean_HT_MT_Html(x))

In [ ]:
from pprint import pprint
pprint(df[['text', 'text_noHT_MT']].head(1)['text'][0])
pprint(df[['text', 'text_noHT_MT']].head(1)['text_noHT_MT'][0])

### Sentiment

In [ ]:
# initialize afinn sentiment analyzer

from afinn import Afinn
af = Afinn()

# compute sentiment scores (polarity) and labels
sentiment_scores = [af.score(tweet) for tweet in df['text_clean'].values]
sentiment_category = ['positive' if score > 0 
                          else 'negative' if score < 0 
                              else 'neutral' 
                                  for score in sentiment_scores]
df['sentiment_score'] = sentiment_scores
df['sentiment_category']  = sentiment_category
print(df.columns)    
# sentiment statistics per news category
df_senti = pd.DataFrame([list(df['text']), list(df['text_clean']), sentiment_scores, sentiment_category]).T
df_senti.columns = ['tweet', 'cleaned_text', 'sentiment_score', 'sentiment_category']
df_senti['sentiment_score'] = df_senti.sentiment_score.astype('float')
df_senti.head(5)